<a href="https://colab.research.google.com/github/ufownl/lap_analysis/blob/main/lap_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lap Analysis

**Lap Analysis** uses the lap details page generated by [stracker](https://github.com/gregsqueeb/S-P-Tracker) to draw a time-difference chart. It is totally open-source under [GPLv3](https://www.gnu.org/licenses/gpl-3.0.txt) and free to use.

## Install

In [ ]:
import os
os.chdir("/content")
!rm -r lap_analysis
!git clone https://github.com/ufownl/lap_analysis.git
os.chdir("/content/lap_analysis")
!pip3 install -r requirements.txt
from lap_analysis import LapDataParser, process_data, align_data, lap_time

## Setup arguments

In [ ]:
class Args():
  def __init__(self, epsilon=1e-4):
    print("Please input URL of lap details page:")
    self.__url = input("> ")
    print("Please input circuit length (km):")
    self.__length = float(input("> "))
    print("Please input lapid of manual selection (press Enter if no manual selection):")
    self.__lapid = input("> ")
    self.__epsilon = epsilon

  @property
  def url(self):
    return self.__url if len(self.__url) > 7 else None

  @property
  def length(self):
    return self.__length

  @property
  def lapid(self):
    return self.__lapid if len(self.__lapid) > 0 else None

  @property
  def epsilon(self):
    return self.__epsilon

args = Args(1e-4)

## Request data and draw chart

In [ ]:
%matplotlib inline

import re
import requests
import matplotlib.pyplot as plt

cookies = None
if not args.lapid is None:
  m = re.match("^\s*(https?://\S+)/", args.url)
  if not m is None:
    r = requests.get(m[1] + "/lapdetails_store_lapid", {
      "lapid": args.lapid
    })
    cookies = r.cookies
r = requests.get(args.url, cookies=cookies)
p = LapDataParser()
p.feed(r.text)
data = tuple((x, y) for x, y in process_data(p.data, args.epsilon))
t = [(x, lap_time(x * args.length * 1000, y)) for x, y in align_data(data, args.epsilon)]
n = min(len(x) for x, _ in t)
fig = plt.figure()
ax0 = fig.add_subplot(111)
ax0.set_xlabel("Track Position (km)")
ax0.set_ylabel("Velocity (km/h)")
for i, (x, y) in enumerate(data):
  ax0.plot(x[:n] * args.length, y[:n], ":", label="serie-%d"%i)
ax0.legend()
ax1 = ax0.twinx()
ax1.set_ylabel("Time Diff (s)")
ax1.plot(t[0][0][:n] * args.length, t[0][1][:n] - t[1][1][:n], "g", label="time diff")
ax1.grid(True)
plt.show()